In [1]:
import os
from functools import reduce
import json                                                                     
import random
from collections import OrderedDict
import pdb
from copy import deepcopy

import pycrfsuite                                                               
import pandas as pd                                                             
import numpy as np                    
#import brick_parser                                                            
#reload(brick_parser)                                                           
#from brick_parser import tagList, tagsetList, equipTagsetList, pointTagsetList, locationTagsetList,equalDict, pointTagList, equipTagList, locationTagList, equipPointDict


In [10]:
building_name = 'ebu3b'
token_type = 'justseparate'
#token_types = ['alphaandnum', 'justseparate']

data_available_buildings = []

if token_type == 'justseparate':
    label_filename = 'metadata/{0}_label_dict.json'.format(building_name)
else:
    label_filename = 'metadata/{0}_label_dict_{1}.json'.format(building_anme, token_type)
with open(label_filename, 'r') as fp:
    label_dict = json.load(fp)
    
with open("metadata/%s_sentence_dict_%s.json" \
          % (building_name, token_type), "r") as fp:
    sentence_dict = json.load(fp)
    
if building_name in data_available_buildings:
    with open("model/fe_%s.json"%building_name, "r") as fp:
        data_feature_dict = json.load(fp)
    with open("model/fe_%s.json"%building_name, "r") as fp:
        normalized_data_feature_dict = json.load(fp)
    for srcid in sentence_dict.keys():
        if not normalized_data_feature_dict.get(srcid):
            normalized_data_feature_dict[srcid] = None

cluster_filename = 'model/%s_word_clustering_%s.json' % (building_name, token_type)
if os.path.isfile(cluster_filename):
    with open(cluster_filename, 'r') as fp:
        cluster_dict = json.load(fp)

In [7]:
'''
feature_num = len(list(data_feature_dict.values())[0])
bin_size = int(len(data_feature_dict)/10)
bin_range_list_list = list()
for i in range(0,feature_num):
    bin_range_list = list()
    sorted_features = sorted([feature[i] for feature in data_feature_dict.values()])
    bin_begin = sorted_features[0]
    bin_begin_idx = 0
    bin_cnt = 1
    for j, feat in enumerate(sorted_features):
        if j==0:
            continue
        elif j > bin_begin_idx+bin_size and sorted_features[j-1]!=feat:
            bin_range_list.append((bin_begin, feat))
            bin_begin = feat
            bin_begin_idx = j
    #for j in range(0,bin_num):
    bin_range_list.append((bin_begin, sorted_features[-1]+1))
    #bin_range_list[-1] = (bin_range_list[-1][0]\
    #                      , bin_range_list[-1][1]) # to apply < consistently.
    bin_range_list_list.append(bin_range_list)

def find_bin(bin_range_list, val):
    for i, bin_range in enumerate(bin_range_list):
        if val >= bin_range[0] and val<bin_range[1]:
            return i
        
normalized_data_feature_dict = dict()
for srcid, features in data_feature_dict.items():
    normalized_data_feature_dict[srcid] = \
        [find_bin(bin_range_list, feat) for bin_range_list, feat \
         in zip(bin_range_list_list, features)]
'''

'\nfeature_num = len(list(data_feature_dict.values())[0])\nbin_size = int(len(data_feature_dict)/10)\nbin_range_list_list = list()\nfor i in range(0,feature_num):\n    bin_range_list = list()\n    sorted_features = sorted([feature[i] for feature in data_feature_dict.values()])\n    bin_begin = sorted_features[0]\n    bin_begin_idx = 0\n    bin_cnt = 1\n    for j, feat in enumerate(sorted_features):\n        if j==0:\n            continue\n        elif j > bin_begin_idx+bin_size and sorted_features[j-1]!=feat:\n            bin_range_list.append((bin_begin, feat))\n            bin_begin = feat\n            bin_begin_idx = j\n    #for j in range(0,bin_num):\n    bin_range_list.append((bin_begin, sorted_features[-1]+1))\n    #bin_range_list[-1] = (bin_range_list[-1][0]    #                      , bin_range_list[-1][1]) # to apply < consistently.\n    bin_range_list_list.append(bin_range_list)\n\ndef find_bin(bin_range_list, val):\n    for i, bin_range in enumerate(bin_range_list):\n       

In [11]:
#def calc_features(sentence,labels):
def calc_features_deprecated(sentence, ts_features=None):
    sentenceFeatures = list()
    for i, word in enumerate(sentence):
        features = [
            'word.lower=' + word.lower(),
            'word.isdigit=%s' % word.isdigit()
        ]
        if i>0:
            features.extend([
                    '-1:word.lower=' + sentence[i-1].lower()
                ])
        else:
            features.append('BOS')
            
        if i<len(sentence)-1:
            features.extend([
                    '+1:word.lower=' + sentence[i+1].lower(),
                ])
        else:
            features.append('EOS')
        sentenceFeatures.append(features)
    return sentenceFeatures

#def calc_features(sentence,labels):
def calc_features(sentence, ts_features=None):
    sentenceFeatures = list()
    for i, word in enumerate(sentence):
        features = {
            'word.lower='+word.lower(): 1.0,
            'word.isdigit': float(word.isdigit())
        }
        if i>0:
            features['-1:word.lower='+sentence[i-1].lower()] = 1.0
        else:
            features['BOS'] = 1.0
            
        if i<len(sentence)-1:
            features['+1:word.lower='+sentence[i+1].lower()] = 1.0
        else:
            features['EOS'] = 1.0
        if ts_features:
            for j, feat in enumerate(ts_features):
                features['ts_feat_'+str(j)] = feat
        sentenceFeatures.append(features)
    return sentenceFeatures

In [12]:
# Learning Sample Selection
USE_CLUSTER = True
SAMPLE_NUM = 200

sample_srcid_list = set()
length_counter = lambda x:len(x[1])
ander = lambda x,y:x and y
labeled_srcid_list = list(label_dict.keys())
if USE_CLUSTER:
    sample_cnt = 0
    sorted_cluster_dict = OrderedDict(sorted(cluster_dict.items(),key=length_counter, reverse=True))
    while len(sample_srcid_list) < SAMPLE_NUM:
        for cluster_num, srcid_list in sorted_cluster_dict.items():
            valid_srcid_list = set(srcid_list).intersection(set(labeled_srcid_list))\
                                .difference(set(sample_srcid_list))
            if len(valid_srcid_list) > 0:
                sample_srcid_list.add(random.choice(list(valid_srcid_list)))
            if len(sample_srcid_list) >= SAMPLE_NUM:
                break
else:
    random_idx_list = random.sample(range(0,len(labeled_srcid_list)),SAMPLE_NUM)
    sample_srcid_list = [labeled_srcid_list[i] for i in random_idx_list]
    

In [13]:
# Cluster counting (remove later)
cluster_counter_dict = dict((cluster_id,0) for cluster_id in cluster_dict.keys())

for srcid in sample_srcid_list:
    for cluster_id, srcid_list in cluster_dict.items():
        if srcid in srcid_list:
            cluster_counter_dict[cluster_id] += 1

In [14]:
#%%time
trainer = pycrfsuite.Trainer(verbose=False, algorithm='pa')#{‘lbfgs’, ‘l2sgd’, ‘ap’, ‘pa’, ‘arow’}
trainer.set_params({'feature.possible_states': True,
                    'feature.possible_transitions':True,                    
                   })
#for srcid, setence in sentence_dict.items():
for srcid in sample_srcid_list:
    labels = label_dict[srcid]
    sentence = sentence_dict[srcid]
    #trainer.append(pycrfsuite.ItemSequence(calc_features(sentence, labels)), labels)
    if building_name in data_available_buildings:
        data_features = normalized_data_feature_dict[srcid]
    else:
        data_features = None
    trainer.append(pycrfsuite.ItemSequence(
        calc_features(sentence, data_features)), labels)

In [15]:
trainer.get_params()

{'averaging': True,
 'c': 1.0,
 'epsilon': 0.0,
 'error_sensitive': True,
 'feature.minfreq': 0.0,
 'feature.possible_states': True,
 'feature.possible_transitions': True,
 'max_iterations': 100,
 'type': 1}

In [16]:
#%%time
trainer.train('random.crfsuite')

In [17]:
tagger = pycrfsuite.Tagger()
tagger.open('random.crfsuite')

In [18]:
#%%time
predicted_dict = dict()
score_dict = dict()
for srcid, sentence in sentence_dict.items():
    if building_name in data_available_buildings:
        data_features = normalized_data_feature_dict[srcid]
    else:
        data_features = None
    predicted = tagger.tag(calc_features(sentence, data_features))
    predicted_dict[srcid] = predicted
    score_dict[srcid] = tagger.probability(predicted)

In [19]:
DEBUG = True
precisionOfTrainingDataset = 0.0                                                
totalWordCount = 0.0                                                            
labeledSrcidList = list(label_dict.keys())
randomSrcidList = sample_srcid_list

result_dict = dict()

#for srcid in label_dict.keys():                                              
for srcid in sentence_dict.keys():
    #if srcid in randomSrcidList:
    #    continue
#for i, srcid in enumerate(sentence_dict.keys()):                                
    #if not i in randIdxList:                                                   
    #    continue                                                               
    if DEBUG:
        if srcid in randomSrcidList:
            print("===================== %s TRAINING ==================== %e" 
                  % (srcid, score_dict[srcid]))
        else:
            print("===================== %s TEST        ================= %e" 
                  % (srcid, score_dict[srcid]))
    sentence = sentence_dict[srcid]                                              
    predicted = predicted_dict[srcid]                                            
    if not srcid in label_dict.keys():                                       
        for word, predTag in zip(sentence, predicted): 
            if DEBUG:
                pass
                #print('{:20s} {:20s}'.format(word,predTag))
            else:
                pass
    else:                                                                       
        printing_pairs = list()                                                 
        for word, predTag, origLabel in zip(sentence, predicted, label_dict[srcid]):
            printing_pair = [word,predTag,origLabel]                            
            
            
            if (building_name=='ebu3b' and not origLabel in ['none', 'identifier', 'network_adapter', 'room'])\
            or (building_name=='soda' and not origLabel in ['site', 'building ahu', 'building chilled/condensor water loop', 'building hot water loop']):
                if origLabel=='soda':
                    pdb.set_trace()
                if predTag==origLabel:                                          
                    precisionOfTrainingDataset += 1                             
                    printing_pair = ['O'] + printing_pair                       
                else:                                                           
                    printing_pair = ['X'] + printing_pair                                       
                totalWordCount += 1                                             
            else:                                                               
                printing_pair = ['O'] + printing_pair                           
            printing_pairs.append(printing_pair)                                
        if 'X' in [pair[0] for pair in printing_pairs]:                         
            for (flag, word, predTag, origLabel) in printing_pairs:
                if DEBUG:
                    print('{:5s} {:20s} {:20s} {:20s}'\
                          .format(flag, word, predTag, origLabel))                                          

print("# Learning Sample: ",len(randomSrcidList))
print("# Test Sample: ",len(label_dict) - len(randomSrcidList))
print("Precision: ", precisionOfTrainingDataset/totalWordCount)

===================== 505_0_3000002 TEST        ================= 5.843156e-58
===================== 505_0_3000003 TEST        ================= 6.377373e-48
===================== 505_0_3000043 TRAINING ==================== 1.767865e-47
===================== 505_0_3000045 TEST        ================= 7.815248e-47
===================== 505_0_3000070 TEST        ================= 4.471430e-49
===================== 505_0_3000073 TEST        ================= 2.171732e-45
===================== 505_0_3000074 TEST        ================= 1.194945e-45
===================== 505_0_3000075 TEST        ================= 2.144066e-49
===================== 505_0_3000076 TEST        ================= 1.605674e-50
===================== 505_0_3000090 TEST        ================= 9.679186e-53
===================== 505_0_3000093 TEST        ================= 2.094067e-43
===================== 505_0_3000094 TEST        ================= 2.110974e-47
===================== 505_0_3000095 TEST        ====

In [20]:
for srcid in randomSrcidList:
    sentence = sentence_dict[srcid]
    if 'i' in sentence:
        print(sentence)